In [1]:
# hide
# default_exp tests.core.test_compose
from nbdev.showdoc import *
from block_types.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Test compose

In [2]:
# export
import pytest
import os
import joblib
from IPython.display import display
import pandas as pd
import numpy as np
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.utils import Bunch
from sklearn.preprocessing import FunctionTransformer

from block_types.core.compose import *
from block_types.core.block_types import Component, PandasComponent, PickleSaverComponent
from block_types.core.utils import PickleIO
from block_types.utils.utils import remove_previous_results
from block_types.core.data_conversion import DataConverter, PandasConverter

import block_types.config.bt_defaults as dflt
from block_types.utils.utils import check_last_part

In [3]:
#export
@pytest.fixture (name='column_transformer_data')
def column_transformer_data_fixture():
    return column_transformer_data()

@pytest.fixture (name='multi_split_data')
def multi_split_data_fixture():
    return multi_split_data()

## Parallel

### `find_last_fitted_model`

#### Using Sequential

##### First test / example

In [4]:
# export
from block_types.utils.dummies import make_pipe_fit1

def test_pipeline_find_last_fitted_model_seq_others ():
    path_results = 'test_pipeline_find_last_fitted_model_seq_start'
    remove_previous_results (path_results=path_results)
    
    # pipelines
    pipe1 = make_pipe_fit1 ()
    X = np.array([1,2,3]).reshape(-1,1)
    r1 = pipe1.fit_apply (X)
    
    # case 1: component A
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted
    pipe2.A.raise_error = True
    with pytest.raises (RuntimeError):
        r2 = pipe2.fit_apply (None)
    remove_previous_results (path_results=path_results)
    
    # case 2: component B
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    r = pipe2.B.fit_apply (r)
    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted
    pipe2.A.raise_error = True
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    remove_previous_results (path_results=path_results)
    
    # case 2: component C
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    r = pipe2.B.fit_apply (r)
    r = pipe2.C.fit_apply (r)
    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted
    pipe2.A.raise_error = True
    pipe2.B.raise_error = True
    pipe2.B.estimator = Bunch ()
    pipe2.C.estimator = Bunch ()
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    remove_previous_results (path_results=path_results)

In [5]:
tst.run (test_pipeline_find_last_fitted_model_seq_others, tag='dummy')

running test_pipeline_find_last_fitted_model_seq_others
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False


applying A (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/A_result.pk
fitting pipeline (using whole data)
applying A (on whole data)


Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False


saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/A_result.pk
fitting B (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/models/B_estimator.pk
applying B (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/B_result.pk
fitting pipeline (using whole data)
fitting B (using whole data)
loading from /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/models/B_estimator.pk
loaded pre-trained B
applying B (on whole data)
loading from /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/B_result.pk
loaded pre-computed result
fitting C (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipel

Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False


applying A (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/A_result.pk
fitting B (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/models/B_estimator.pk
applying B (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/B_result.pk
fitting C (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/models/C_estimator.pk
applying C (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/C_result.pk
fitting pipeline (using whole data)
loading from /home/jcidatascience/jaume/workspace/remote/temp/block-types/test_pipeline_find_last_fitted_model_seq_start/models/B_estimat

Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False


##### Second test / example

#### Using Parallel

In [6]:
# export
from block_types.utils.dummies import make_pipe_fit2

def test_pipeline_find_last_fitted_model_parallel_2 ():
    path_results = 'test_pipeline_find_last_fitted_model_parallel_2'
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    # pipelines
    pipe1 = make_pipe_fit2 ()
    X = np.array([1,2,3]).reshape(-1,1)
    r1 = pipe1.fit_apply (X)
    
    # ******************************************************
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    b1 = pipe2.obj.B1.fit_apply (r)
    b2 = pipe2.obj.B2.fit_apply (r)
    
    b3a = pipe2.obj.B3a.fit_apply (r)
    b3b = pipe2.obj.B3b.fit_apply (b3a)
    b3c = pipe2.obj.B3c.fit_apply (b3b)
    b3d = pipe2.obj.B3d.fit_apply (b3c)
    
    b4a = pipe2.obj.B4a.fit_apply (r)
    b4b = pipe2.obj.B4b.fit_apply (b4a)
    b4c = pipe2.obj.B4c.fit_apply (b4b)
    b4d = pipe2.obj.B4d.fit_apply (b4c)
    b4e = pipe2.obj.B4e.fit_apply (b4d)
    
    b5 = pipe2.obj.B5.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    
    pipe2.logger.info (f'\n{"*"*100}\n{"*"*100}\n')
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    b1 = pipe2.obj.B1.fit_apply (r)
    b2 = pipe2.obj.B2.fit_apply (r)
    
    b3a = pipe2.obj.B3a.fit_apply (r)
    b3b = pipe2.obj.B3b.fit_apply (b3a)
    b3c = pipe2.obj.B3c.fit_apply (b3b)
    b3d = pipe2.obj.B3d.fit_apply (b3c)
    
    b4 = pipe2.obj.pipeline_1_1.fit_apply (r)
    b5 = pipe2.obj.B5.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    
    pipe2.logger.info (f'\n{"*"*100}\n{"*"*100}\n')
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    r = pipe2.obj.parallel.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B1.raise_error = True
    pipe2.obj.B2.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
    pipe2.obj.B3d.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    pipe2.obj.B4e.raise_error = True
    
    pipe2.obj.B5.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    pipe2.logger.info (f'\n{"*"*100}\n{"*"*100}\n')
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True, new_parallel=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    r = pipe2.obj.new_parallel.fit_apply (r)
    r = pipe2.C.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B1.raise_error = True
    pipe2.obj.B2.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
    pipe2.obj.B3d.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    pipe2.obj.B4e.raise_error = True
    
    pipe2.obj.B5.raise_error = True
    
    pipe2.new_parallel.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)
    
    # ******************************************************
    pipe2.logger.info (f'\n{"*"*100}\n{"*"*100}\n')
    pipe2 = make_pipe_fit2 (path_results=path_results, verbose=2, root=True, new_parallel=True)
    # second
    r = pipe2.A0.fit_apply (X)
    r = pipe2.A1.fit_apply (r)
    r = pipe2.obj.new_parallel.fit_apply (r)
    r = pipe2.C.fit_apply (r)
    r = pipe2.D.fit_apply (r)

    all_fitted = pipe2.find_last_fitted_model ()
    assert all_fitted

    pipe2.A0.raise_error = True
    pipe2.A1.raise_error = True
    
    pipe2.obj.B1.raise_error = True
    pipe2.obj.B2.raise_error = True
    
    pipe2.obj.B3a.raise_error = True
    pipe2.obj.B3b.raise_error = True
    pipe2.obj.B3c.raise_error = True
    pipe2.obj.B3d.raise_error = True
        
    pipe2.obj.B4a.raise_error = True
    pipe2.obj.B4b.raise_error = True
    pipe2.obj.B4c.raise_error = True
    pipe2.obj.B4d.raise_error = True
    pipe2.obj.B4e.raise_error = True
    
    pipe2.obj.B5.raise_error = True
    
    pipe2.new_parallel.raise_error = True
    pipe2.C.raise_error = True
    
    pipe2.A1.create_estimator()
    pipe2.obj.B2.create_estimator()
    pipe2.obj.B3a.create_estimator()
    pipe2.obj.B3b.create_estimator()
    pipe2.obj.B4b.create_estimator()
    pipe2.obj.B4c.create_estimator()
    pipe2.obj.B4e.create_estimator()
    pipe2.obj.B5.create_estimator()
    
    pipe2.D.create_estimator()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    
    remove_previous_results (path_results=path_results)

In [7]:
tst.run (test_pipeline_find_last_fitted_model_parallel_2, tag='dummy')

running test_pipeline_find_last_fitted_model_parallel_2
parallel class: <class 'block_types.core.compose.Parallel'>
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False
Did not find y as separate argument, but no_labels is False


TypeError: _apply() takes 2 positional arguments but 15 were given

## Data conversion

In [ ]:
#export
from block_types.utils.dummies import (DataSource, SumXY, MaxOfPositiveWithSeparateLabels, Sum1direct,
                                       Multiply10direct, subtract_xy, MinOfPositiveWithoutSeparateLabels)

In [ ]:
result = DataSource ().apply()

In [ ]:
result[0]

In [ ]:
result[1]

In [ ]:
result

In [ ]:
SumXY ().fit_apply (result)

In [ ]:
#%debug

In [ ]:
class SumXYConverter (DataConverter):
    def convert_before_transforming (self, X, **kwargs):
        self.label = X[2]
        return X[0], X[1]
    def convert_after_transforming (self, result, **kwargs):
        result['label'] = self.label
        return result

In [ ]:
pipe = Sequential (DataSource (),
                   SumXY (data_converter=SumXYConverter),
                   PandasComponent (apply=lambda X: X*2))
result = pipe ()
result

In [ ]:
result2 = pipe.fit_apply (())
result2

In [ ]:
result3=MaxOfPositiveWithSeparateLabels (data_converter='PandasConverter').fit_apply (result2)
result3

In [ ]:
parallel = Parallel (Sum1direct (data_converter='PandasConverter'), 
                     Multiply10direct (data_converter='PandasConverter'),
                     finalize_result=lambda X: tuple(X)
                    )
result4=parallel.fit_apply (result3)
result4

In [ ]:
result5=PandasComponent(apply=subtract_xy).fit_apply (result4)
result5

In [ ]:
result6=MinOfPositiveWithoutSeparateLabels (
    data_converter=PandasConverter (separate_labels=False)).fit_apply (result5)
result6

In [ ]:
pipe = Sequential (DataSource (),
                   SumXY (data_converter=SumXYConverter),
                   PandasComponent (apply=lambda X: X*2),
                   MaxOfPositiveWithSeparateLabels (data_converter='PandasConverter'),
                   Parallel (Sum1direct (data_converter='PandasConverter'), 
                             Multiply10direct (data_converter='PandasConverter'),
                             finalize_result=lambda X: tuple(X)),
                   PandasComponent(apply=subtract_xy),
                   MinOfPositiveWithoutSeparateLabels (),
                   make_column_transformer ((Multiply10direct (), ['a','b']), 
                                            (Sum1direct (), ['c','d'])),
                   Sum1direct ())